# Part 1

In [1]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install panda

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install fancyimpute

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import Imputer  
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingRegressor#create a dictionary of our models

##from fancyimpute import MICE

from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split


%matplotlib inline

In [7]:
import helpers

In [8]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV #Importing RandomizedSearchCV for random forest regressor

In [10]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)

In [44]:
train_df.head(5).transpose()

rental_id,7236931,7331920,7139224,7364827,7242151
addr_unit,#22A,#406,#18A,#23E,#3B
building_id,551248,3373,70747,42076,319156
bedrooms,4,0,3,1,3
bathrooms,3,1,3,1,1
size_sqft,1300,475,1886,827,1200
created_at,2018-06-04 16:31:06,2018-07-03 20:11:43,2018-05-02 16:10:27,2018-07-13 16:06:55,2018-06-05 16:35:40
addr_street,645 OCEAN AVENUE,93 WORTH STREET,305 EAST 51 STREET,70 WEST 37TH STREET,43 MAC DONOUGH STREET
addr_city,Brooklyn,New York,New York,New York,Brooklyn
addr_zip,11226,10013,10022,10018,11216
addr_lat,40.647,40.7166,40.7554,40.7513,40.6811


#### Build training data

In [47]:
# extract usable features
feature_cols = [
    'bedrooms', 'bathrooms', 'floor_count', 'size_sqft',
    'has_washer_dryer', 'has_roofdeck', 'has_doorman', 'has_dishwasher']
train_features = train_df[feature_cols] 

# impute missing values with imputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(train_features)
imp.transform(train_features)
#LOOK INTO IMPUTING USING MICE-- see fancyimpute

# construct target vector
train_target = train_df['rent']

#### Fit models for training data

In [48]:
lreg = LinearRegression()
lreg.fit(train_features, train_target)

rf = RandomForestRegressor()
rf.fit(train_features, train_target)

br = BaggingRegressor(tree.DecisionTreeRegressor(random_state=1))
br.fit(train_features, train_target)

gb = GradientBoostingRegressor()
gb.fit(train_features, train_target)

ab = AdaBoostRegressor()
ab.fit(train_features, train_target)

knn = KNeighborsRegressor()#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=55)#fit model to training data
knn_gs.fit(train_features, train_target)

estimators=[('knn', knn_gs), ('rf', rf), ('lreg', lreg),('br',br),('ab',ab)]
voter = VotingRegressor(estimators)
voter.fit(train_features, train_target)



/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


VotingRegressor(estimators=[('knn',
                             GridSearchCV(cv=55,
                                          error_score='raise-deprecating',
                                          estimator=KNeighborsRegressor(algorithm='auto',
                                                                        leaf_size=30,
                                                                        metric='minkowski',
                                                                        metric_params=None,
                                                                        n_jobs=None,
                                                                        n_neighbors=5,
                                                                        p=2,
                                                                        weights='uniform'),
                                          iid='warn', n_jobs=None,
                                          param_grid={'n_neighbors': arr

In [59]:
knn = KNeighborsRegressor()#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=50)#fit model to training data
knn_gs.fit(train_features, train_target)

GridSearchCV(cv=50, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [60]:
pprint(knn.get_params()) #Can test more KNN hyper paramters as well as the other models

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}


In [61]:
#pprint(rf.get_params()) #Checking hyper parameters of rf

#Creating our random grid to sample during fitting
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [62]:
rrf = RandomForestRegressor() #Base RFR for use in our RandomizedSCV
rf_random = RandomizedSearchCV(estimator = rrf, 
                               param_distributions = random_grid, 
                               n_iter = 10, cv = 3, verbose = 2,
                               random_state = 42, n_jobs = -1)

rf_random.fit(train_features, train_target)



Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.0min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [63]:
rf_random.best_params_   #Looking at best params

{'n_estimators': 2000,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 50,
 'bootstrap': True}

In [64]:
best_random = rf_random.best_estimator_   #Setting new RSCV using best parameters
best_random.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=2000,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

#### Predict and Measure Using Test 1

In [65]:
test_features = test_df[feature_cols] 

# impute missing values with medians
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(test_features)
imp.transform(test_features)

# construct predictions 
test_df['predicted'] = rf.predict(test_features)
print("MSE_rf", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)

test_df['predicted'] = gb.predict(test_features)
print("MSE_gb", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)

test_df['predicted'] = ab.predict(test_features)
print("MSE_ab", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)

test_df['predicted'] = br.predict(test_features)
print("MSE_br", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)

test_df['predicted'] = lreg.predict(test_features)
print("MSE_lreg", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)

test_df['predicted'] = knn_gs.predict(test_features)
print("MSE_knn", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)

test_df['predicted'] = voter.predict(test_features)
print("MSE_voter", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)


MSE_rf 570364.2794636443
MSE_gb 2133050.236497565
MSE_ab 5835605.475630233
MSE_br 447217.7302967674
MSE_lreg 3680742.7922809115
MSE_knn 3197265.023415816
MSE_voter 1347268.2668490182


#### Predict and Measure Using Test 1 (Randomized Search CV)

In [66]:
test_df['predicted'] = rf_random.predict(test_features)   #Testing randomized search CV
print("MSE_RSCV", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)

test_df['predicted'] = best_random.predict(test_features)   #Testing randomized search CV
print("MSE_Best_RSCV", mean_squared_error(test_df['rent'], test_df['predicted']))
test_df = test_df.drop('predicted', 1)


MSE_RSCV 2290575.5521978405
MSE_Best_RSCV 2290575.5521978405


#### Combine Data, Predict Values for Test 2

In [67]:
master_df = train_df.append(test_df, sort=False)

# impute missing values with imputer
#imp = SimpleImputer(missing_values=np.nan, strategy='median')
#imp.fit(master_features)
#imp.transform(master_features)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)

master_target = master_df['rent']

rf.fit(master_features, master_target)
gb.fit(master_features, master_target)
ab.fit(master_features, master_target)
br.fit(master_features, master_target)
lreg.fit(master_features, master_target)

params_knn = {'n_neighbors': np.arange(1, 30)}#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=20)#fit model to training data
knn_gs.fit(master_features, master_target)

estimators=[('knn', knn_gs), ('rf', rf), ('lreg', lreg),('br',br),('ab',ab),('best_random',best_random)]
voter = VotingRegressor(estimators)
voter.fit(master_features, master_target)

VotingRegressor(estimators=[('knn',
                             GridSearchCV(cv=20,
                                          error_score='raise-deprecating',
                                          estimator=KNeighborsRegressor(algorithm='auto',
                                                                        leaf_size=30,
                                                                        metric='minkowski',
                                                                        metric_params=None,
                                                                        n_jobs=None,
                                                                        n_neighbors=5,
                                                                        p=2,
                                                                        weights='uniform'),
                                          iid='warn', n_jobs=None,
                                          param_grid={'n_neighbors': arr

#### Randomized Search CV for test 2 (Without best parameters)

In [68]:
#rf_random.fit(master_features, master_target)
best_random.fit(master_features, master_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=2000,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

#### Randomized Search CV (test2)

In [69]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = rf_random.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_rf.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_rf: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = best_random.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_rf.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_best_rf: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_rf:  6796516.256468636
MVE_best_rf:  6796516.256468636


#### Random Forest Regressor (test2)

In [70]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = rf.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_rf.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_rf: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_rf:  7469921.004313083


#### GradientBoosting Regressor (test2)

In [71]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = rf.predict(submit2_features)
submit2_df['predictions'].to_csv('sample_submission2.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_gb: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_gb:  7469921.004313083


#### ABoost Regressor (test2)

In [72]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = ab.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_ab.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_ab: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_ab:  12070655.83057286


#### Bagged Regressor (test2)

In [73]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = br.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_br.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_br: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_br:  7779442.2744454695


#### Linear Regression (test2)

In [78]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = lreg.predict(submit2_features)
submit2_df['predictions'].to_csv('sample_submission_lreg.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_lreg: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_lreg:  4765059.741542864


#### KNN Regressor (test2)

In [77]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = knn_gs.predict(submit2_features)
submit2_df['predictions'].to_csv('sample_submission_knn.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_knn: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))


MVE_knn:  5121476.26826033


### Voting Regressor (test2)

In [76]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = voter.predict(submit2_features)
submit2_df['predictions'].to_csv('submission_vote.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
print("MVE_vote: ", mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent']))

MVE_vote:  6096437.819029653


## Attempt to use categorical data with our models

In [ ]:
#Will look into one hot encoding and labeling
